### Imports

In [1]:
import numpy as np
import pandas as pd
np.seterr(over='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

### Neural Network class

Taken from [here](https://github.com/ludobouan/pure-numpy-feedfowardNN/blob/master/FeedforwardNN.py)

In [61]:
class NeuralNetwork():
    def __init__(self):
        np.random.seed(1)  
        self.weights = {}  
        self.num_layers = 1
        self.adjustments = {}

    def add_layer(self, shape):
        # Create weights with shape specified + biases
        self.weights[self.num_layers] = np.vstack((2 * np.random.random(shape) - 1, 2 * np.random.random((1, shape[1])) - 1))
        
        # Initialize the adjustements for these weights to zero
        self.adjustments[self.num_layers] = np.zeros(shape)
        self.num_layers += 1

    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, data):
        
        # Pass data through pretrained network
        for layer in range(1, self.num_layers+1):
            data = np.dot(data, self.weights[layer-1][:, :-1]) + self.weights[layer-1][:, -1] + self.biases[layer]
            data = self.__sigmoid(data)
        return data

    def __forward_propagate(self, data):
        # Progapagate through network and hold values for use in back-propagation
        activation_values = {}
        activation_values[1] = data
        for layer in range(2, self.num_layers+1):
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][-1, :].T + self.biases[layer]
            data = self.__sigmoid(data).T
            activation_values[layer] = data
        return activation_values

    def simple_error(self, outputs, targets):
        return targets - outputs

    def sum_squared_error(self, outputs, targets):
        return 0.5 * np.mean(np.sum(np.power(outputs - targets, 2), axis=1))

    def __back_propagate(self, output, target):
        deltas = {}
        # Delta of output Layer
        deltas[self.num_layers] = output[self.num_layers] - target

        # Delta of hidden Layers
        for layer in reversed(range(2, self.num_layers)):  # All layers except input/output
            a_val = output[layer]
            weights = self.weights[layer][:-1, :]
            prev_deltas = deltas[layer+1]
            deltas[layer] = np.multiply(np.dot(weights, prev_deltas), self.__sigmoid_derivative(a_val))

        # Caclculate total adjustements based on deltas
        for layer in range(1, self.num_layers):
            self.adjustments[layer] += np.dot(deltas[layer+1], output[layer].T).T

    def __gradient_descente(self, batch_size, learning_rate):
        # Calculate partial derivative and take a step in that direction
        for layer in range(1, self.num_layers):
            partial_d = (1/batch_size) * self.adjustments[layer]
            self.weights[layer][:-1, :] += learning_rate * -partial_d
            self.weights[layer][-1, :]  += learning_rate*1e-3 * -partial_d[-1, :]


    def train(self, inputs, targets, epochs=1, learning_rate=1, stop_accuracy=1e-5, verbose=None):
        error = []
        for iteration in range(epochs):
            for i in range(len(inputs)):
                x = inputs[i]
                y = targets[i]
                # Pass the training set through our neural network
                output = self.__forward_propagate(x)

                # Calculate the error
                loss = self.sum_squared_error(output[self.num_layers], y)
                error.append(loss)

                # Calculate Adjustements
                self.__back_propagate(output, y)
                
                if verbose and (i+1) % verbose == 0:
                    print(f"Error is {loss}")

            self.__gradient_descente(i, learning_rate)

            # Check if accuarcy criterion is satisfied
            if np.mean(error[-(i+1):]) < stop_accuracy and iteration > 0:
                break

        return(np.asarray(error), iteration+1)

### XOR Function 

In [62]:
# Create instance of a neural network
nn = NeuralNetwork()

# Add Layers (Input layer is created by default)
nn.add_layer((2, 9))
nn.add_layer((9, 1))

# XOR function
training_data = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]]).reshape(4, 2, 1)
training_labels = np.asarray([[0], [1], [1], [0]])

error, iteration = nn.train(training_data, training_labels, epochs=5000)
print('Error = ', np.mean(error[-4:]))
print('Epoches needed to train = ', iteration)

AttributeError: 'NeuralNetwork' object has no attribute 'biases'

### MNIST

In [57]:
# Create instance of a neural network
nn = NeuralNetwork()

# Add Layers (Input layer is created by default)
nn.add_layer((784, 500))
nn.add_layer((500, 200))
nn.add_layer((200, 10))

train   = pd.read_csv("data/train.csv").sample(frac=0.1).reset_index(drop=True)
train_y = train.label.values.reshape(len(train), 1)
train_x = train.drop("label", axis=1).values.reshape(len(train), 784, 1)

In [58]:
error, iteration = nn.train(train_x, train_y, num_epochs=3, learning_rate=0.01, verbose=4200)
print('Error = ', np.mean(error[-4:]))
print('Epoches needed to train = ', iteration)

Error is 0.18989375040183312
Error is 0.04981591124726583
Error is 8.212667056620322e-07
Error =  14.1262503526
Epoches needed to train =  3
